Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [16]:
import numpy as np
from icecream import ic

In [17]:
class Solution:
    
    def __init__(self, n_knapsacks, n_items, n_dimensions, values, weights, constraints):
        self.n_knapsacks = n_knapsacks
        self.n_items = n_items
        self.n_dimensions = n_dimensions
        self.values = values
        self.weights = weights
        self.constraints = constraints
        self.penalty_weight = values.max() * 2
        
    # Check that the same object does not appear in multiple knapsacks
    def checkSameObject(self, solution, index):
        return np.any(solution[:, index])
    '''
    # Check if the solution is valid
    def is_valid(self, solution, knapsack):
        items = solution[knapsack, :]
        tot_weights = self.weights[items, :].sum(axis=0)
        #True if no constraint is exceeded
        return np.all(tot_weights <= self.constraints[knapsack, :])
    '''
    def fitness(self, solution):
        total_value = 0  # sum of values of objects of all knapsacks 
        penalty = 0 

        for k in range(self.n_knapsacks):
            items = solution[k, :]
            total_value += self.values[items].sum()

            # penalty if it exceeds the limits
            tot_w = self.weights[items, :].sum(axis=0)
            over = np.maximum(tot_w - self.constraints[k, :], 0)
            penalty += over.sum() * 10  # proportional penalty

        # penalty if one object is in more knapsacks
        if np.any(solution.sum(axis=0) > 1):
            penalty += self.penalty_weight

        return total_value - penalty
    
    def tweak(self, solution):
        k = np.random.randint(0, self.n_knapsacks)
        i = np.random.randint(0, self.n_items)

        # if object already assigned, remove it
        # otherwise, put it in knapsack k
        if self.checkSameObject(solution, i):
            solution[:, i] = False
        else:
            solution[k, i] = True
    
    def resolve(self, max_steps, initial_temp, cooling_rate, verbose = True):
        # Initialize with empty solution
        solution = np.zeros((self.n_knapsacks, self.n_items), dtype=bool)
        
        # SA parameters
        T = initial_temp
        
        current_fitness = self.fitness(solution)
        best_fitness = current_fitness
        best_solution = solution.copy()

        for step in range(max_steps):
            # save previous state 
            old_solution = solution.copy()
            old_fitness = current_fitness
            
            # add/remove random object
            self.tweak(solution)

            new_fitness = self.fitness(solution)
            delta = new_fitness - old_fitness

            #SA
            if delta > 0 or np.random.random() < np.exp(delta / T):
                current_fitness = new_fitness
                if new_fitness > best_fitness:
                    best_fitness = new_fitness
                    best_solution = solution.copy()
            else:
                # restore previous state
                solution[:] = old_solution
                current_fitness = old_fitness

            # T gradually drops
            T *= cooling_rate

            # debug
            if verbose and step % 100 == 0:
                print(f"Step: {step:4d} | Best_fitness: {best_fitness:6d} | T: {T:.4f}")

        print("\n=== FINAL RESULT ===")
        print(f"Best value found: {best_fitness}")
        print(f"Best configuration:\n{best_solution}")

## TEST PROBLEMS

In [18]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

print("=== PROBLEM 1 ===")
solver = Solution(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)
solver.resolve(max_steps=5000, initial_temp=10.0, cooling_rate=0.99)


=== PROBLEM 1 ===
Step:    0 | Best_fitness:     52 | T: 9.9000
Step:  100 | Best_fitness:    936 | T: 3.6237
Step:  200 | Best_fitness:    988 | T: 1.3264
Step:  300 | Best_fitness:   1065 | T: 0.4855
Step:  400 | Best_fitness:   1065 | T: 0.1777
Step:  500 | Best_fitness:   1065 | T: 0.0650
Step:  600 | Best_fitness:   1065 | T: 0.0238
Step:  700 | Best_fitness:   1065 | T: 0.0087
Step:  800 | Best_fitness:   1065 | T: 0.0032
Step:  900 | Best_fitness:   1065 | T: 0.0012
Step: 1000 | Best_fitness:   1065 | T: 0.0004
Step: 1100 | Best_fitness:   1065 | T: 0.0002
Step: 1200 | Best_fitness:   1065 | T: 0.0001
Step: 1300 | Best_fitness:   1065 | T: 0.0000
Step: 1400 | Best_fitness:   1065 | T: 0.0000
Step: 1500 | Best_fitness:   1065 | T: 0.0000
Step: 1600 | Best_fitness:   1065 | T: 0.0000
Step: 1700 | Best_fitness:   1065 | T: 0.0000
Step: 1800 | Best_fitness:   1065 | T: 0.0000
Step: 1900 | Best_fitness:   1065 | T: 0.0000
Step: 2000 | Best_fitness:   1065 | T: 0.0000
Step: 2100 | Bes

In [19]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

print("=== PROBLEM 2 ===")
solver = Solution(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)
solver.resolve(max_steps=5000, initial_temp=10.0, cooling_rate=0.99)

=== PROBLEM 2 ===
Step:    0 | Best_fitness:    967 | T: 9.9000
Step:  100 | Best_fitness:  23704 | T: 3.6237
Step:  200 | Best_fitness:  27264 | T: 1.3264
Step:  300 | Best_fitness:  29944 | T: 0.4855
Step:  400 | Best_fitness:  31005 | T: 0.1777
Step:  500 | Best_fitness:  32212 | T: 0.0650
Step:  600 | Best_fitness:  32245 | T: 0.0238
Step:  700 | Best_fitness:  32245 | T: 0.0087
Step:  800 | Best_fitness:  32875 | T: 0.0032
Step:  900 | Best_fitness:  32875 | T: 0.0012
Step: 1000 | Best_fitness:  32875 | T: 0.0004
Step: 1100 | Best_fitness:  32875 | T: 0.0002
Step: 1200 | Best_fitness:  32875 | T: 0.0001
Step: 1300 | Best_fitness:  32875 | T: 0.0000
Step: 1400 | Best_fitness:  32875 | T: 0.0000
Step: 1500 | Best_fitness:  32875 | T: 0.0000
Step: 1600 | Best_fitness:  32875 | T: 0.0000
Step: 1700 | Best_fitness:  32875 | T: 0.0000
Step: 1800 | Best_fitness:  32875 | T: 0.0000
Step: 1900 | Best_fitness:  32875 | T: 0.0000
Step: 2000 | Best_fitness:  32875 | T: 0.0000
Step: 2100 | Bes

In [20]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

print("=== PROBLEM 3 ===")
solver = Solution(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)
solver.resolve(max_steps=10000, initial_temp=10.0, cooling_rate=0.99)


=== PROBLEM 3 ===
Step:    0 | Best_fitness:    616 | T: 9.9000
Step:  100 | Best_fitness:  50661 | T: 3.6237
Step:  200 | Best_fitness:  96074 | T: 1.3264
Step:  300 | Best_fitness: 137774 | T: 0.4855
Step:  400 | Best_fitness: 180583 | T: 0.1777
Step:  500 | Best_fitness: 220685 | T: 0.0650
Step:  600 | Best_fitness: 268576 | T: 0.0238
Step:  700 | Best_fitness: 315129 | T: 0.0087
Step:  800 | Best_fitness: 355140 | T: 0.0032
Step:  900 | Best_fitness: 395371 | T: 0.0012
Step: 1000 | Best_fitness: 444269 | T: 0.0004
Step: 1100 | Best_fitness: 492038 | T: 0.0002
Step: 1200 | Best_fitness: 529593 | T: 0.0001
Step: 1300 | Best_fitness: 569978 | T: 0.0000
Step: 1400 | Best_fitness: 600504 | T: 0.0000
Step: 1500 | Best_fitness: 639744 | T: 0.0000
Step: 1600 | Best_fitness: 667768 | T: 0.0000
Step: 1700 | Best_fitness: 700334 | T: 0.0000
Step: 1800 | Best_fitness: 738496 | T: 0.0000
Step: 1900 | Best_fitness: 770431 | T: 0.0000
Step: 2000 | Best_fitness: 797998 | T: 0.0000
Step: 2100 | Bes